## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps.datasets
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bairiki,KI,1.3292,172.9752,87.80,66,40,14.97,scattered clouds
1,1,Sinnamary,GF,5.3833,-52.9500,76.42,91,90,9.26,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,-1.68,50,20,3.44,few clouds
3,3,Mahebourg,MU,-20.4081,57.7000,76.24,100,75,29.93,moderate rain
4,4,Ostrovnoy,RU,68.0531,39.5131,10.71,95,85,11.92,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bairiki,KI,1.3292,172.9752,87.80,66,40,14.97,scattered clouds
17,17,Hithadhoo,MV,-0.6000,73.0833,82.96,70,2,8.70,clear sky
23,23,Manaus,BR,-3.1019,-60.0250,81.09,83,75,3.44,broken clouds
31,31,Kapaa,US,22.0752,-159.3190,82.38,89,75,18.41,light rain
33,33,Butaritari,KI,3.0707,172.7902,82.35,72,28,21.36,scattered clouds
44,44,Georgetown,MY,5.4112,100.3354,87.73,77,20,4.61,few clouds
47,47,Olinda,BR,-8.0089,-34.8553,83.93,70,40,8.05,scattered clouds
49,49,Vaini,TO,-21.2000,-175.2000,84.36,79,75,4.61,broken clouds
60,60,Tautira,PF,-17.7333,-149.1500,81.91,74,80,9.69,light rain
67,67,Gizo,SB,-8.1030,156.8419,80.24,81,100,9.86,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows

preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                99
City                   99
Country                99
Lat                    99
Lng                    99
Max Temp               99
Humidity               99
Cloudiness             99
Wind Speed             99
Current Description    99
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bairiki,KI,87.80,scattered clouds,1.3292,172.9752,
17,Hithadhoo,MV,82.96,clear sky,-0.6000,73.0833,
23,Manaus,BR,81.09,broken clouds,-3.1019,-60.0250,
31,Kapaa,US,82.38,light rain,22.0752,-159.3190,
33,Butaritari,KI,82.35,scattered clouds,3.0707,172.7902,
44,Georgetown,MY,87.73,few clouds,5.4112,100.3354,
47,Olinda,BR,83.93,scattered clouds,-8.0089,-34.8553,
49,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
60,Tautira,PF,81.91,light rain,-17.7333,-149.1500,
67,Gizo,SB,80.24,overcast clouds,-8.1030,156.8419,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    lodging = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        
        hotel_df.loc[index, "Hotel Name"] = lodging["results"][0]["name"]
    
    except (IndexError):
        
        print("Hotel not found")
        

hotel_df.head(10)
        

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bairiki,KI,87.80,scattered clouds,1.3292,172.9752,Tarawa Boutique Hotel
17,Hithadhoo,MV,82.96,clear sky,-0.6000,73.0833,Scoop Guest House
23,Manaus,BR,81.09,broken clouds,-3.1019,-60.0250,Manaus Hotéis Millennium
31,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
33,Butaritari,KI,82.35,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
44,Georgetown,MY,87.73,few clouds,5.4112,100.3354,Cititel Penang
47,Olinda,BR,83.93,scattered clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
49,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
60,Tautira,PF,81.91,light rain,-17.7333,-149.1500,Pension Oaoa
67,Gizo,SB,80.24,overcast clouds,-8.1030,156.8419,Imagination Island


In [9]:
# 7. Drop the rows where there is no Hotel Name.
#Replace Empty String with 'NaN'
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bairiki,KI,87.80,scattered clouds,1.3292,172.9752,Tarawa Boutique Hotel
17,Hithadhoo,MV,82.96,clear sky,-0.6000,73.0833,Scoop Guest House
23,Manaus,BR,81.09,broken clouds,-3.1019,-60.0250,Manaus Hotéis Millennium
31,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
33,Butaritari,KI,82.35,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
676,Kolondieba,ML,87.82,overcast clouds,11.0882,-6.8926,Hotel Dakan
685,Meulaboh,ID,81.28,overcast clouds,4.1363,96.1285,Kost Pria
706,Gorom-Gorom,BF,82.72,overcast clouds,14.4439,-0.2361,NaN
715,Bannu,PK,81.88,clear sky,32.9854,70.6027,Oxford Hostel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
#Remove 'NaN values'
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bairiki,KI,87.80,scattered clouds,1.3292,172.9752,Tarawa Boutique Hotel
17,Hithadhoo,MV,82.96,clear sky,-0.6000,73.0833,Scoop Guest House
23,Manaus,BR,81.09,broken clouds,-3.1019,-60.0250,Manaus Hotéis Millennium
31,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
33,Butaritari,KI,82.35,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
663,Kirakira,SB,84.56,overcast clouds,-10.4544,161.9205,SanBiz Lodge
676,Kolondieba,ML,87.82,overcast clouds,11.0882,-6.8926,Hotel Dakan
685,Meulaboh,ID,81.28,overcast clouds,4.1363,96.1285,Kost Pria
715,Bannu,PK,81.88,clear sky,32.9854,70.6027,Oxford Hostel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))